Hunting for fgdc in the iso (or missing structures and the reasons)

In [15]:
%reload_ext autoreload
%autoreload 2

import json as js  # name conflict with sqla
import sqlalchemy as sqla
from sqlalchemy.orm import sessionmaker

from mpp.models import Response

from lxml import etree

In [16]:
def _extract_tag(t):
    if not t:
        return
    return t.split('}')[-1]

def _taggify(e):
    tags = [e.tag] + [m.tag for m in e.iterancestors()]
    tags.reverse()

    try:
        return [_extract_tag(t) for t in tags]
    except:
        return []

def extract_comments(xml):
    comment_elems = xml.xpath('//*[comment()]')
    
    for comment_elem in comment_elems:
        comment_tag = '/'.join(_taggify(comment_elem))
        comments = '\n'.join([x.text for x in comment_elem.iter(tag=etree.Comment)])
        
        yield comment_tag, comments

In [17]:
# grab the clean text from the rds
with open('../local/big_rds.conf', 'r') as f:
    conf = js.loads(f.read())

# our connection
engine = sqla.create_engine(conf.get('connection'))
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

In [18]:
sketchy_sql = '''with i
as (
    select d.response_id, jsonb_array_elements(d.identity::jsonb) ident
    from identities d
    where d.identity is not null
)

select r.id, r.cleaned_content
from responses r join i on i.response_id = r.id
where i.ident->>'protocol' = 'ISO'
limit %s
offset %s;
'''

In [13]:
response = session.query(Response).filter(Response.source_url=='http://catalog.data.gov/harvest/object/d037f002-d13c-45d2-bd57-d59c5822a14a').first()

xml = etree.fromstring(response.cleaned_content.encode('utf-8'))
comments = [(t, c) for t, c in extract_comments(xml)]

In [14]:
comments

[('MI_Metadata/identificationInfo/MD_DataIdentification/citation/CI_Citation',
  'ORIGIN'),
 ('MI_Metadata/contentInfo',
  ' translated from eainfo to MD_FeatureCatalogueDescription')]

In [21]:
session.rollback()

In [24]:
with open('outputs/iso_with_comments.txt', 'w') as f:
    f.write('[\n')

# s, e, i = 0, 10, 10
s, e, i = 0, 19689, 100

for x in xrange(s, e, i):
    for response in session.execute(sketchy_sql % (i, x)):
        
        try:
            xml = etree.fromstring(response['cleaned_content'].encode('utf-8'))
        except:
            print 'failed xml', response['id']
            continue
        
        comments = [(t, c) for t, c in extract_comments(xml)]
        
        if comments:
            with open('outputs/iso_with_comments.txt', 'a') as f:
                f.write(js.dumps([{"id": response['id'], "tags": t, "comments": c} for t, c in comments]) + ',\n')

                
with open('outputs/iso_with_comments.txt', 'a') as f:
    f.write(']')

failed xml 145560
failed xml 183293
failed xml 196865
failed xml 219701
failed xml 223566
failed xml 252783
failed xml 307810
failed xml 351247
failed xml 402936
failed xml 453490
failed xml 503992
failed xml 539074
failed xml 563466
failed xml 576196
failed xml 653347
failed xml 667256
failed xml 721563
failed xml 722226


Deleted the bit of code, but I think an invalid namespace for an attribute.

failed xml 145560
failed xml 183293
failed xml 196865
failed xml 219701
failed xml 223566
failed xml 252783
failed xml 307810
failed xml 351247
failed xml 402936
failed xml 453490
failed xml 503992
failed xml 539074
failed xml 563466
failed xml 576196
failed xml 653347
failed xml 667256
failed xml 721563
failed xml 722226